In [7]:
# @title imports

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from skimage.segmentation import slic, mark_boundaries

import torchvision
import torchvision.models as models

import math

import os

from torch.utils.data import random_split, DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from torchvision.datasets import MNIST, CIFAR100
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

import pickle

import matplotlib.pyplot as plt

from skimage.segmentation import slic
from skimage.measure import regionprops
from skimage import filters, graph
import torch
import numpy as np
from torch_geometric.utils.convert import from_networkx
from torchvision.transforms import Resize

In [8]:
transform = Compose([
    Resize((32, 32)),
    ToTensor(),
    Normalize(0, 1),
])
dataset = CIFAR100(root="../data/image/CIFAR100", download=True, transform=transform)

Files already downloaded and verified


Code

In [11]:
data = pickle.load(open("../data/image/CIFAR100/graph/SLIC_16_15.pkl", "rb"))

In [13]:
from torch_geometric.loader import DataLoader as PyGDataLoader

loader = PyGDataLoader(data, batch_size=32, shuffle=True)

In [15]:
b = next(iter(loader))

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 9 but got size 5 for tensor number 1 in the list.